In [18]:
from datetime import date
import pandas as pd
from collections import defaultdict
from typing import Optional
from pydantic import BaseModel, Field

In [46]:
def split_name_id(name):
    # raw_name -> (name, id)
    if pd.isna(name):
        return '', ''
    if '(' in name and ')' in name:
        row_id = name[name.rindex('(') + 1: name.rindex(')')]
        name = name[: name.index('(')]
    else:
        row_id = name
    return name.strip(), row_id.strip()


def _handle_video_action_adjust_df(df):
    """
    1.业务漏斗：
        - 模版展示率: show cnt (113) / user count
        - 模版点击率: click_cnt (120) / user count
        - 模版使用率: use_cnt(220) / user count
        - 相册转化率: 420 / user count
        - 编辑页导出率: 320 + 321 / user count
        - 结果页展示率: 511 / user count
    2.核心指标：
        - 模版下载成功率:  / user count (todo)
        - 模版导出成功率: 330 + 340 / user count

        将 ad_action 的行值转为列，并重命名
        docs: https://docs.google.com/spreadsheets/d/17nSYlxNjQG-uuTEJihCpS7NZZOSBC0w85oNDpiOFLzg/edit#gid=734237912    # noqa
    """

    rename_columns = {
        '110': 'act_show',
        '120': 'act_click',
        '130': 'act_loading_up_scroll',
        '131': 'act_loading_up_scroll_s',
        '140': 'act_loading_down_scroll',
        '141': 'act_loading_down_scroll_s',
        '210': 'act_preview_enter',
        '220': 'act_template_use',
        '221': 'act_preview_favorite',
        '223': 'act_preview_share',
        '226': 'act_preview_download',
        '300': 'act_edit_enter',
        '310': 'act_edit_show_s',
        '320': 'act_edit_export',
        '321': 'act_edit_export_no',
        '322': 'act_edit_return',
        '323': 'act_edit_watermark_remove',
        '325': 'act_edit_resolution_click',
        '330': 'act_edit_export_s',
        '331': 'act_edit_export_f',
        '340': 'act_edit_export_no_s',
        '341': 'act_edit_export_no_f',
        '350': 'act_edit_music_swap',
        '360': 'act_edit_photo_swap',
        '410': 'act_photo_enter',
        '411': 'act_photo_album_swap',
        '412': 'act_photo_scan_f',
        '420': 'act_photo_next',
        '510': 'act_result_enter',
        '511': 'act_result_show',
        '520': 'act_result_home_back',
        '530': 'act_result_whatsapp',
        '531': 'act_result_instagram',
        '532': 'act_result_youtube',
        '533': 'act_result_facebook',
        '540': 'act_result_recommend',
        '550': 'act_result_create_again',
        '551': 'act_result_hd_click',
    }

    class AdActionAdjustCallbackDimension(BaseModel):
        event_date: date
        inst_date: date
        # project_id: str
        datasource: str
        pkg: str
        os: str
        cv: str
        user_id: str
        template_id: Optional[str]
        utm_country: str
        utm_source: str
        utm_campaign_id: str
        utm_campaign_name: str
        utm_medium_id: str
        utm_medium_name: str
        utm_creative_id: str
        utm_creative_name: str

    class AdActionAdjustCallbackMetrics(BaseModel):
        act_show: Optional[int] = Field(default=0)
        act_click: Optional[int] = Field(default=0)
        act_loading_up_scroll: Optional[int] = Field(default=0)
        act_loading_up_scroll_s: Optional[int] = Field(default=0)
        act_loading_down_scroll: Optional[int] = Field(default=0)
        act_loading_down_scroll_s: Optional[int] = Field(default=0)
        act_preview_enter: Optional[int] = Field(default=0)
        act_template_use: Optional[int] = Field(default=0)
        act_preview_favorite: Optional[int] = Field(default=0)
        act_preview_share: Optional[int] = Field(default=0)
        act_preview_download: Optional[int] = Field(default=0)
        act_edit_enter: Optional[int] = Field(default=0)
        act_edit_show_s: Optional[int] = Field(default=0)
        act_edit_export: Optional[int] = Field(default=0)
        act_edit_export_no: Optional[int] = Field(default=0)
        act_edit_return: Optional[int] = Field(default=0)
        act_edit_watermark_remove: Optional[int] = Field(default=0)
        act_edit_resolution_click: Optional[int] = Field(default=0)
        act_edit_export_s: Optional[int] = Field(default=0)
        act_edit_export_f: Optional[int] = Field(default=0)
        act_edit_export_no_s: Optional[int] = Field(default=0)
        act_edit_export_no_f: Optional[int] = Field(default=0)
        act_edit_music_swap: Optional[int] = Field(default=0)
        act_edit_photo_swap: Optional[int] = Field(default=0)
        act_photo_enter: Optional[int] = Field(default=0)
        act_photo_album_swap: Optional[int] = Field(default=0)
        act_photo_scan_f: Optional[int] = Field(default=0)
        act_photo_next: Optional[int] = Field(default=0)
        act_result_enter: Optional[int] = Field(default=0)
        act_result_show: Optional[int] = Field(default=0)
        act_result_home_back: Optional[int] = Field(default=0)
        act_result_whatsapp: Optional[int] = Field(default=0)
        act_result_instagram: Optional[int] = Field(default=0)
        act_result_youtube: Optional[int] = Field(default=0)
        act_result_facebook: Optional[int] = Field(default=0)
        act_result_recommend: Optional[int] = Field(default=0)
        act_result_create_again: Optional[int] = Field(default=0)
        act_result_hd_click: Optional[int] = Field(default=0)

    class AdActionAdjustCallbackRecord(AdActionAdjustCallbackDimension, AdActionAdjustCallbackMetrics):
        pass

    mapping = defaultdict(dict)
    event_keys = [
        'edit_action', 'prev_action', 'tpl_action', 'result_action',
        'ud_result_info', 'result_watermark', 'photo_action'
    ]
    for ind, row in df.iterrows():
        template_id = row.template_id or ''
        for template_id in template_id.split(","):
            # 三种ID: 4 chars, 32 chars, comma sep string
            key = (
                row.event_date, row.inst_date, row.pkg, row.cv, row.utm_country, row.utm_source, row.utm_campaign,
                row.utm_medium, row.utm_creative, row.user_id, template_id
            )
            for event_key in event_keys:
                event_no = str(row[event_key])
                if rename_columns.get(event_no):
                    mapping[key][rename_columns[event_no]] = row.eventCount
    data = list()
    for row_key in mapping:
        event_date, inst_date, pkg, cv, utm_country, utm_source, campaign_name, adgroup_name, creative_name, \
            user_id, template_id = row_key
        metric_d = mapping[row_key]
        utm_campaign_id, utm_campaign_name = split_name_id(campaign_name)
        utm_medium_id, utm_medium_name = split_name_id(adgroup_name)
        utm_creative_id, utm_creative_name = split_name_id(creative_name)
        record = AdActionAdjustCallbackRecord(
            event_date=event_date,
            inst_date=inst_date,
            datasource='adjust',
            pkg="Catcut",
            os="Android",
            cv=cv,
            user_id=user_id,
            template_id=template_id,
            utm_country=utm_country.upper(),
            utm_source=utm_source,
            utm_campaign_id=utm_campaign_id,
            utm_campaign_name=utm_campaign_name,
            utm_medium_id=utm_medium_id,
            utm_medium_name=utm_medium_name,
            utm_creative_id=utm_creative_id,
            utm_creative_name=utm_creative_name,
            **metric_d
        )
        data.append(record.dict())
    return pd.DataFrame(data)

In [47]:
df = pd.read_csv('./bquxjob_3b3baca_183fa469019.csv')
df

,event_date,inst_date,pkg,utm_source,utm_campaign,utm_medium,utm_creative,utm_country,cv,user_id,template_id,edit_action,prev_action,tpl_action,result_action,ud_result_info,result_watermark,photo_action,eventCount
0,2022-10-20,2022-10-19,com.video.music.editor.maker,Organic,NaN,NaN,NaN,ae,1.4.24,2640537fc22b21635a09002b38daed827318d69e,aebe,NaN,NaN,120,NaN,NaN,NaN,NaN,5
1,2022-10-20,2022-09-24,com.video.music.editor.maker,TikTok,catcut_中东_WSY_0930 (1745363009158177),0930_catcut_中东_WSY_优秀素材 (1745363010721809),catcut_in_220920_liaoluhui_1X1_boy04.mp4_002 (...,ae,1.4.24,ce0c3a8b5f0d48d151d987c8c29fd394dfe2fe57,8a81,NaN,NaN,120,NaN,NaN,NaN,NaN,5
2,2022-10-20,2022-10-11,com.video.music.editor.maker,TikTok,catcut_gcc_出量_wl_02_0922 (1744665743244305),catcut_gcc_出量_wl_02_0922 (1744665736607762),pk_210830_chinweiyu_1X1_vido16.mp4_004 (174466...,ae,1.4.24,5ad60c99eb3ba9a1c63d7d7e11fc631feb6e58f2,ceee,NaN,NaN,120,NaN,NaN,NaN,NaN,4
3,2022-10-20,2022-10-20,com.video.music.editor.maker,Unattributed,NaN,NaN,NaN,ae,1.4.24,9515e6717592bb3e7af572482f372c49417a8b50,6b2e,NaN,NaN,120,NaN,NaN,NaN,NaN,3
4,2022-10-20,2022-09-24,com.video.music.editor.maker,TikTok,catcut_中东_WSY_0930 (1745363009158177),0930_catcut_中东_WSY_优秀素材 (1745363010721809),catcut_in_220920_liaoluhui_1X1_boy04.mp4_002 (...,ae,1.4.24,ce0c3a8b5f0d48d151d987c8c29fd394dfe2fe57,9f5d34795cef4f3285e97e45fe02c7e3,NaN,NaN,120,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2022-10-20,2022-10-20,com.video.music.editor.maker,TikTok,catcut_ae_cp1-5_wl_02_1010 (1746294937353249),catcut_ae_cp1-5_wl_02_1010 (1746294943790098),catcut_in_220920_liaoluhui_1X1_couple3.mp4_001...,ae,1.4.24,427e84482f2765c45a88575b4f377d5dded513e1,296cb3868ef2476ba85689dde918eb4a,NaN,NaN,120,NaN,NaN,NaN,NaN,1
116,2022-10-20,2022-10-20,com.video.music.editor.maker,TikTok,catcut_中东_WSY_0930 (1745363009158177),0930_catcut_中东_WSY_优秀素材 (1745363010721809),catcut_in_220920_liaoluhui_1X1_boy05.mp4_005 (...,ae,1.4.24,c8afbbab895ca9bf1e586bfa968ac6d370a579bb,2a2b4f212a5b4a4e85bd80484c62502a,NaN,NaN,120,NaN,NaN,NaN,NaN,1
117,2022-10-20,2022-10-12,com.video.music.editor.maker,Untrusted Devices,Anonymous IPs,NaN,NaN,ae,1.4.24,9fc231e70550d077fcf3cd411e5137ec22415ce4,1894,NaN,NaN,120,NaN,NaN,NaN,NaN,1
118,2022-10-20,2022-10-20,com.video.music.editor.maker,TikTok,catcut_gcc_13迭代_spc_wl_1013 (1746568453358610),catcut_gcc_13迭代_spc_wl_1013 (1746568453358626),in_221013_zhangguiping_1X1_vido1.mp4_001 (1746...,ae,1.4.24,bb40a29d2d026c1ee739fa80eb42d972b0daf24d,3969bde9ca4f45b2bbeb4eaec5bcdab6,NaN,NaN,120,NaN,NaN,NaN,NaN,1


In [48]:
adj_df = _handle_video_action_adjust_df(df)
adj_df

,act_show,act_click,act_loading_up_scroll,act_loading_up_scroll_s,act_loading_down_scroll,act_loading_down_scroll_s,act_preview_enter,act_template_use,act_preview_favorite,act_preview_share,...,user_id,template_id,utm_country,utm_source,utm_campaign_id,utm_campaign_name,utm_medium_id,utm_medium_name,utm_creative_id,utm_creative_name
0,0,5,0,0,0,0,0,0,0,0,...,2640537fc22b21635a09002b38daed827318d69e,aebe,AE,Organic,,,,,,
1,0,5,0,0,0,0,0,0,0,0,...,ce0c3a8b5f0d48d151d987c8c29fd394dfe2fe57,8a81,AE,TikTok,catcut_中东_WSY_0930,1745363009158177,0930_catcut_中东_WSY_优秀素材,1745363010721809,catcut_in_220920_liaoluhui_1X1_boy04.mp4_002,1745363019792402
2,0,4,0,0,0,0,0,0,0,0,...,5ad60c99eb3ba9a1c63d7d7e11fc631feb6e58f2,ceee,AE,TikTok,catcut_gcc_出量_wl_02_0922,1744665743244305,catcut_gcc_出量_wl_02_0922,1744665736607762,pk_210830_chinweiyu_1X1_vido16.mp4_004,1744665748789297
3,0,3,0,0,0,0,0,0,0,0,...,9515e6717592bb3e7af572482f372c49417a8b50,6b2e,AE,Unattributed,,,,,,
4,0,3,0,0,0,0,0,0,0,0,...,ce0c3a8b5f0d48d151d987c8c29fd394dfe2fe57,9f5d34795cef4f3285e97e45fe02c7e3,AE,TikTok,catcut_中东_WSY_0930,1745363009158177,0930_catcut_中东_WSY_优秀素材,1745363010721809,catcut_in_220920_liaoluhui_1X1_boy04.mp4_002,1745363019792402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,1,0,0,0,0,0,0,0,0,...,427e84482f2765c45a88575b4f377d5dded513e1,296cb3868ef2476ba85689dde918eb4a,AE,TikTok,catcut_ae_cp1-5_wl_02_1010,1746294937353249,catcut_ae_cp1-5_wl_02_1010,1746294943790098,catcut_in_220920_liaoluhui_1X1_couple3.mp4_001,1746295047880721
116,0,1,0,0,0,0,0,0,0,0,...,c8afbbab895ca9bf1e586bfa968ac6d370a579bb,2a2b4f212a5b4a4e85bd80484c62502a,AE,TikTok,catcut_中东_WSY_0930,1745363009158177,0930_catcut_中东_WSY_优秀素材,1745363010721809,catcut_in_220920_liaoluhui_1X1_boy05.mp4_005,1745363019796514
117,0,1,0,0,0,0,0,0,0,0,...,9fc231e70550d077fcf3cd411e5137ec22415ce4,1894,AE,Untrusted Devices,Anonymous IPs,Anonymous IPs,,,,
118,0,1,0,0,0,0,0,0,0,0,...,bb40a29d2d026c1ee739fa80eb42d972b0daf24d,3969bde9ca4f45b2bbeb4eaec5bcdab6,AE,TikTok,catcut_gcc_13迭代_spc_wl_1013,1746568453358610,catcut_gcc_13迭代_spc_wl_1013,1746568453358626,in_221013_zhangguiping_1X1_vido1.mp4_001,1746568454520834


In [49]:
adj_df.act_click.sum()

143